In [ ]:
import SimpleITK as sitk
import os 
import torch
from torchvision import transforms
import torchstain
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
## This code is for registering images together 

In [ ]:
#path to RGB images and scala/vasculature labels
path = "/Users/aseemjain/Documents/Research/Otopathology/Cochlea_image_newcompressed/"
scalapath = "/Users/aseemjain/Documents/Research/Otopathology/scalaimagesouput/"
vaspath="/Users/aseemjain/Documents/Research/Otopathology/vascularimages/"

In [ ]:

##Code for registration using simple elastix
fixedimage = sitk.ReadImage(path+"stria_"+str(1)+"_0000.png",sitk.sitkInt8)
fixedscala = sitk.ReadImage(scalapath+"stria_"+str(1)+".png",sitk.sitkInt8)
fixedvasc = sitk.ReadImage(vaspath+"stria_"+str(1)+"_0000.png",sitk.sitkInt8)
#fixedimage=fixedscala
for x in range(2,110):
    movingimage = sitk.ReadImage(path+"stria_"+str(x)+"_0000.png",sitk.sitkInt8)
    movingscala = sitk.ReadImage(scalapath+"stria_"+str(x)+".png",sitk.sitkInt8)
    movingvas = sitk.ReadImage(vaspath+"stria_"+str(x)+"_0000.png",sitk.sitkInt8)

    selx = sitk.ElastixImageFilter()
    selx.SetMovingImage(movingimage)
    #selx.SetParameterMap(selx.GetDefaultParameterMap('affine'))
    parameterMapVector = sitk.VectorOfParameterMap()
    parameterMapVector.append(sitk.GetDefaultParameterMap("affine")) # First do an affine transform
    parameterMapVector.append(sitk.GetDefaultParameterMap("rigid"))  # Then do a rigid transform for improved alignment
    selx.SetParameterMap(parameterMapVector)

    selx.SetFixedImage(fixedimage)
    selx.Execute()
    fixedimage=selx.GetResultImage()
    #nda = sitk.GetArrayFromImage(fixedimage)
    #imgplot = plt.imshow(nda)
    transformixImageFilter = sitk.TransformixImageFilter()
    transformixImageFilter.SetTransformParameterMap(selx.GetTransformParameterMap())

    transformixImageFilter.SetMovingImage(movingscala)
    transformixImageFilter.Execute()
    sitk.WriteImage(transformixImageFilter.GetResultImage(), scalapath+"stria_reg"+str(x)+".tiff")

    transformixImageFilter.SetMovingImage(movingvas)
    transformixImageFilter.Execute()
    sitk.WriteImage(transformixImageFilter.GetResultImage(), vaspath+"vas_reg"+str(x)+".tiff")